**INSTALANDO AS BIBLIOTECAS**

In [ ]:
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px

**BAIXANSO RECURSO NLTK** #Facilita a compreensão de palavras escritas por diversos usuários

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

**LER AQUIVO JSON**

In [ ]:
url = 'https://raw.githubusercontent.com/Hackathon-ONE-II/SentimentAPI/main/data-science/datasets/reviews_mercadolivre_com_br_1.json'
df = pd.read_json(url)  # Atribui á uma variável os dados importados
df.head()  # Visualizar os dados

In [ ]:
df = df.rename(columns={
    'content': 'review_text',
    'rating': 'rating'
})

**Criar o rótulo (sentimento)**

👉 Baseline binário

In [ ]:
df['sentimento'] = df['rating'].apply(
    lambda x: 1 if x >= 4 else 0
)

1 → Positivo

0 → Negativo

**Separar treino e teste**

In [ ]:
from sklearn.model_selection import train_test_split
X = df['review_text'].astype(str)
y = df['sentimento']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

**Vetorização com TF-IDF (ESSENCIAL)**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords', quiet=True)
stop_words = nltk.corpus.stopwords.words('portuguese')

tfidf = TfidfVectorizer(
    stop_words=stop_words,
    max_features=5000
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

**Treinar o modelo simples**

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

**Avaliar o baseline**

In [ ]:
from sklearn.metrics import classification_report

y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

**Teste rápido (demo)**

In [ ]:
exemplos = [
    "Produto excelente, recomendo!",
    "Entrega atrasou e atendimento péssimo"
]

ex_tfidf = tfidf.transform(exemplos)
model.predict(ex_tfidf)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import nltk

nltk.download('stopwords', quiet=True)
stop_words = nltk.corpus.stopwords.words('portuguese')

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        stop_words=stop_words,
        max_features=5000
    )),
    ('clf', LogisticRegression(max_iter=1000))
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

**Distribuição das Avaliações**

In [ ]:
plt.figure()
df['rating'].value_counts().sort_index().plot(kind='bar')
plt.title("Distribuição das Avaliações")
plt.xlabel("Nota")
plt.ylabel("Quantidade")
plt.show()

**TAMANHO DOS COMENTÁRIOS**

In [ ]:
df['review_length'] = df['review_text'].astype(str).str.len()
df['review_length'].describe()

**PALAVRAS MAIS FREQUENTES**

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

# Ensure nltk stopwords is downloaded
nltk.download('stopwords', quiet=True)

stop_words = set(nltk.corpus.stopwords.words('portuguese'))

def tokenize_and_clean(text):
    # Ensure text is string, convert if not (e.g., if there are NaNs)
    text = str(text).lower() # Convert to lowercase
    tokens = word_tokenize(text, language='portuguese')
    # Remove stopwords and non-alphabetic tokens
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

df['tokens'] = df['review_text'].apply(tokenize_and_clean)

all_words = [word for tokens_list in df['tokens'] for word in tokens_list]
Counter(all_words).most_common(20)

**NUVEM DE PALAVRAS**

In [ ]:
from wordcloud import WordCloud

text = " ".join([word for tokens_list in df['tokens'].dropna() for word in tokens_list])

wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    stopwords=stop_words
).generate(text)

plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

**NOTA X TAMANHO DO COMENTÁRIO**

In [ ]:
plt.figure()
sns.boxplot(x='rating', y='review_length', data=df)
plt.title("Nota x Tamanho do Comentário")
plt.show()

**CLASSIFICAÇÃO SIMPLES DO SENTIMENTO**

In [ ]:
df['sentimento'] = df['rating'].apply(
    lambda x: 'positivo' if x >= 4 else 'negativo'
)

df['sentimento'].value_counts()

**EXEMPLOS DE REVIEWS NEGATIVAS (PARA INSIGHT)**

In [ ]:
df[df['sentimento'] == 'negativo'][['rating', 'review_text']].head(5)

**Salvar o baseline**

In [ ]:
import joblib
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk

nltk.download('stopwords', quiet=True)
stop_words = nltk.corpus.stopwords.words('portuguese')

# Redefine and fit the pipeline for robustness in this saving cell
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        stop_words=stop_words,
        max_features=5000
    )),
    ('clf', LogisticRegression(max_iter=1000))
])
pipeline.fit(X_train, y_train)

joblib.dump(pipeline, "baseline_tfidf.joblib")